# Análise de API's (Open FDA)

### Importando bibliotecas:

In [74]:
import pandas as pd
import duckdb
import httpx
import pandera.pandas as pa
from pandera.typing import Series
import pyarrow
import duckdb

### Estruturando os dados:

In [83]:
# Buscando os dados
response = httpx.get('https://api.fda.gov/drug/event.json?search=serious:1&limit=10')
data = response.json()

In [84]:
# Estruturando os dados
df = pd.json_normalize(data['results'])
df.head()

,safetyreportid,transmissiondateformat,transmissiondate,serious,seriousnessdeath,receivedateformat,receivedate,receiptdateformat,receiptdate,fulfillexpeditecriteria,...,sender.sendertype,receiver.receivertype,receiver.receiverorganization,seriousnessother,occurcountry,seriousnesshospitalization,patient.patientagegroup,patient.summary.narrativeincludeclinical,seriousnesslifethreatening,patient.patientweight
0,5801206-7,102,20090109,1,1,102,20080707,102,20080625,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10003300,102,20141002,1,NaN,102,20140306,102,20140306,2,...,2,6,FDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10003301,102,20141002,1,NaN,102,20140228,102,20140228,2,...,2,6,FDA,1,NaN,NaN,NaN,NaN,NaN,NaN
3,10003311,102,20151125,1,NaN,102,20140312,102,20150812,2,...,2,6,FDA,NaN,US,1,6,CASE EVENT DATE: 20120601,NaN,NaN
4,10003312,102,20141212,1,NaN,102,20140312,102,20140417,1,...,2,6,FDA,1,US,1,NaN,NaN,1,41.8


In [85]:
# Analisando os dados:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 39 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   safetyreportid                               10 non-null     object 
 1   transmissiondateformat                       10 non-null     object 
 2   transmissiondate                             10 non-null     object 
 3   serious                                      10 non-null     object 
 4   seriousnessdeath                             3 non-null      object 
 5   receivedateformat                            10 non-null     object 
 6   receivedate                                  10 non-null     object 
 7   receiptdateformat                            10 non-null     object 
 8   receiptdate                                  10 non-null     object 
 9   fulfillexpeditecriteria                      10 non-null     object 
 10  compa

### Criando contrato de dados (Pandera)

In [92]:
schema = pa.DataFrameSchema(
    columns={
        "serious": pa.Column(int, coerce=True),
        "receiver.receiverorganization": pa.Column(str, nullable=True),
        "patient.patientsex": pa.Column(int, nullable=True, coerce=True)
    },
    strict="filter" #Ignora colunas que não estão no schema
)

In [93]:
# Validando o dataframe:
validated_df = schema.validate(df)
print(validated_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   serious                        10 non-null     int64 
 1   patient.patientsex             10 non-null     int64 
 2   receiver.receiverorganization  9 non-null      object
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes
None


### Salvando os resultados em .parquet

In [73]:
folder = '/home/jovyan/work/Data/OpenFDA/'
validated_df.to_parquet(f"{folder}adverse_events.parquet", engine='pyarrow')
print('Arquivo .parquet Salvo')

Arquivo .parquet Salvo


### Lendo os resultados com DuckDB

In [95]:
dados = '/home/jovyan/work/Data/OpenFDA/adverse_events.parquet'

duckdb.sql(f"""

Select * from read_parquet('{dados}')

""").df()

,serious,patient.patientsex,receiver.receiverorganization
0,1,1,None
1,1,2,FDA
2,1,2,FDA
3,1,2,FDA
4,1,2,FDA
5,1,1,FDA
6,1,1,FDA
7,1,2,FDA
8,1,2,FDA
9,1,1,FDA
